In [11]:
import pandas as pd
import scipy as sc
import numpy as np
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import seaborn as sb


In [12]:
knesset21eclections = pd.read_excel(r"Knesset Elections\-21-.xlsx")
knesset21eclections = knesset21eclections.drop("מספר קלפי", axis=1)
knesset21eclections = knesset21eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()


knesset20eclections = pd.read_excel(r"Knesset Elections\-20-.xlsx")
knesset20eclections = knesset20eclections.drop("מספר קלפי", axis=1)
knesset20eclections = knesset20eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()

knesset19eclections = pd.read_excel(r"Knesset Elections\-19-.xlsx")
knesset19eclections = knesset19eclections.drop("מספר קלפי", axis=1)
knesset19eclections = knesset19eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()

knesset18eclections = pd.read_excel(r"Knesset Elections\-18-.xlsx")
knesset18eclections = knesset18eclections.drop("סמל קלפי", axis=1)
knesset18eclections = knesset18eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()

knesset17eclections = pd.read_excel(r"Knesset Elections\-17-2006.xls")
knesset17eclections = knesset17eclections.drop("מספר קלפי", axis=1)
knesset17eclections = knesset17eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()


knesset16eclections = pd.read_excel(r"Knesset Elections\-16-2003.xls")
knesset16eclections = knesset16eclections.drop("סמל קלפי", axis=1)
knesset16eclections = knesset16eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()


knesset15eclections = pd.read_excel(r"Knesset Elections\-15-1999-.xls")
knesset15eclections = knesset15eclections.drop("מס' רץ", axis=1).drop("קלפי", axis=1)
knesset15eclections = knesset15eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()


knesset14eclections = pd.read_excel(r"Knesset Elections\-14-1996-.xls")
knesset14eclections = knesset14eclections.drop("סמל קלפי", axis=1).drop("כתובת", axis=1)
knesset14eclections = knesset14eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()



In [14]:
knesset17eclections.head()
knesset18eclections.head()
knesset19eclections.head()
knesset21eclections

,,בזב,מצביעים,פסולים,כשרים,אמת,ג,דעם,ום,ז,זי,...,צק,ץ,ץז,ץי,ק,קי,קן,קף,ר,שס
שם ישוב,סמל ישוב,,,,,,,,,,,,,,,,,,,,,
אבו ג'ווייעד (שבט),967,1915,275,5,270,5,0,175,13,1,0,...,0,0,0,0,0,2,1,0,0,0
אבו גוש,472,4707,1915,72,1843,35,2,368,490,2,1,...,1,1,0,0,1,1,1,1,6,30
אבו סנאן,473,9234,4845,82,4763,60,6,592,2152,2,0,...,37,3,0,4,6,7,1,7,12,62
אבו עבדון (שבט),958,138,12,0,12,0,0,2,3,0,0,...,0,0,0,0,0,0,0,0,1,0
אבו קורינאת (שבט),968,2927,640,11,629,8,0,433,34,0,0,...,0,1,0,0,1,2,1,3,2,2
אבו קרינאת (יישוב),1342,543,213,4,209,0,0,82,24,0,0,...,0,0,0,0,0,0,0,1,0,2
אבו רובייעה (שבט),966,3919,761,8,753,24,0,517,70,2,0,...,4,0,0,1,1,0,2,0,2,4
אבו רוקייק (שבט),961,4691,1129,13,1116,2,3,1007,18,1,0,...,1,2,0,2,0,2,0,0,2,3
אבו תלול,1375,867,390,4,386,0,0,355,10,0,0,...,1,0,0,0,0,0,0,0,1,1


In [17]:

boharim_percentage14 = knesset14eclections['ס"ה מצביעים']/knesset14eclections['ס"ה בוחרים']
boharim_percentage14.sort_values(ascending=False)

שם ישוב            סמל ישוב
משמר איילון        670         1.105839
נצרים              5408        1.085714
ורד יריחו          3639        1.029703
כפר דרום           5405        1.011765
גן נר              1274        1.007890
אבני חפץ           3793        0.991736
חגי                3764        0.988506
אלמוג              3556        0.982143
יסודות             440         0.975728
כרמי צור           3766        0.975410
ח'ואלד             986         0.972332
כלנית              1229        0.970149
מגדל עוז           3561        0.967213
דולב               3747        0.960784
שקף                1233        0.956044
בית אל ב'          3714        0.951515
סוסיה              3756        0.950980
כפר טרומן          673         0.947917
איתמר              3762        0.946903
עוקבי )בנו עוקבה(  957         0.946058
טלמון              3788        0.946043
לפידות             1173        0.944444
אעבלין             529         0.943904
כפר קאסם           634         0.941522
מתתיהו      

In [22]:
psulim_percentage = knesset19eclections["פסולים"]/knesset19eclections["מצביעים"]
print(psulim_percentage.sort_values(ascending=False))
print("\n\n")

print("תל אביב", psulim_percentage["תל אביב - יפו"])
print("\n\n")
print("ירושלים",psulim_percentage["ירושלים"])

שם ישוב            סמל ישוב
שער מנשה           921         0.090909
אבו קרינאת (יישוב  1342        0.084746
עין נקובא          521         0.065466
רומת הייב          997         0.065445
ארגמן              3598        0.058824
כרם שלום           1085        0.052632
תראבין א-צאנע (שב  970         0.051020
אבו ג'ווייעד (שבט  967         0.050691
עין חוד            1320        0.046296
עין השלושה         676         0.045685
דמיידה             1317        0.045455
אבו גוש            472         0.042842
ערב אל נעים        1335        0.042735
ירכא               502         0.038418
שחר                7           0.038217
מוקייבלה           635         0.036087
ג'נאביב (שבט)      976         0.035088
טובא-זנגריה        962         0.033031
טייבה (בעמק)       497         0.032000
כמאנה              1331        0.031390
קלע                4024        0.030769
מגאר               481         0.030487
עין ראפה           514         0.030418
מסעדה              4203        0.030303
עילוט       